<a href="https://colab.research.google.com/github/vupadhayayula/Speech_Processing_LTRC/blob/main/Emotion_recognition_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess

cp: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 97% 417M/428M [00:02<00:00, 160MB/s]
100% 428M/428M [00:03<00:00, 149MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/toronto-emotional-speech-set-tess.zip')
zip_ref.extractall('/content/drive/MyDrive/emotion')
zip_ref.close()

In [ ]:
import glob
voice = glob.glob('/content/drive/MyDrive/emotion/TESS Toronto emotional speech set data/**/*.wav',recursive=True)
print(len(voice))

2800


In [ ]:
import librosa
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
os.listdir("/content/drive/MyDrive/emotion/TESS Toronto emotional speech set data")

['OAF_Fear',
 'OAF_Pleasant_surprise',
 'OAF_Sad',
 'OAF_angry',
 'OAF_disgust',
 'OAF_happy',
 'OAF_neutral',
 'YAF_angry',
 'YAF_disgust',
 'YAF_fear',
 'YAF_happy',
 'YAF_neutral',
 'YAF_pleasant_surprised',
 'YAF_sad']

In [ ]:
parent_dir = "/content/drive/MyDrive/emotion/TESS Toronto emotional speech set data"
speaker_folders = [
    "OAF_Fear",
    "OAF_Pleasant_surprise",
    'OAF_Sad',
    'OAF_angry',
    'OAF_disgust',
    'OAF_happy',
    'OAF_neutral',
    'YAF_angry',
    'YAF_disgust',
    'YAF_fear',
    'YAF_happy',
    'YAF_neutral',
    'YAF_pleasant_surprised',
    'YAF_sad'
]

In [ ]:
import numpy as np
import os
import librosa

def extract_features(parent_dir, speaker_folders):
    features = []
    labels = []

    for i, speaker_folder in enumerate(speaker_folders):
        speaker_folder_path = os.path.join(parent_dir, speaker_folder)
        for filename in os.listdir(speaker_folder_path):
            if filename.endswith(".wav"):
                file_path = os.path.join(speaker_folder_path, filename)
                audio, sr = librosa.load(file_path)
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=16)
                features.append(mfccs.T)
                labels.append(i)
    max_length = max(f.shape[0] for f in features)

    padded_features = [np.pad(f, ((0, max_length - f.shape[0]), (0, 0)), mode='constant') for f in features]

    return np.array(padded_features), np.array(labels)

# Extract features and labels
X, y = extract_features(parent_dir, speaker_folders)


from tensorflow import keras
from sklearn.model_selection import train_test_split

# Assuming X and y are already defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

# Reshape X_train, X_validation, and X_test for LSTM input
timesteps = X_train.shape[1]  # Number of time steps
features = X_train.shape[2]   # Number of features

X_train = X_train.reshape(-1, timesteps, features)
X_validation = X_validation.reshape(-1, timesteps, features)
X_test = X_test.reshape(-1, timesteps, features)

# Define the model
model = keras.Sequential()
model.add(keras.layers.LSTM(64, input_shape=(timesteps, features), return_sequences=True))
model.add(keras.layers.LSTM(64))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(14, activation='softmax'))

# Compile the model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Define early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train,
                    validation_data=(X_validation, y_validation),
                    batch_size=32,
                    epochs=60,
                    callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 129, 64)           20736     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 14)                910       
                                                                 
Total params: 58830 (229.80 KB)
Trainable params: 58830 (229.80 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/60
56/56 [

In [ ]:
model.save("emotion_recognition_lstm.keras")

In [ ]:
# Define the model
model = keras.Sequential()
model.add(keras.layers.LSTM(64, input_shape=(timesteps, features), return_sequences=True))
model.add(keras.layers.LSTM(64, return_sequences=True))  # Additional LSTM layer
model.add(keras.layers.LSTM(64))  # Additional LSTM layer
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(32, activation='relu'))  # Additional Dense layer
model.add(keras.layers.Dropout(0.1))  # Additional Dropout layer
model.add(keras.layers.Dense(14, activation='softmax'))

# Compile the model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()
# Define early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train,
                    validation_data=(X_validation, y_validation),
                    batch_size=32,
                    epochs=60,
                    callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 129, 64)           20736     
                                                                 
 lstm_3 (LSTM)               (None, 129, 64)           33024     
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)               

In [ ]:
model.save("")

In [ ]:
import librosa
import numpy as np
from tensorflow import keras

path = "/content/drive/MyDrive/emotion/TESS Toronto emotional speech set data/YAF_angry/YAF_bar_angry.wav"
audio, sr=librosa.load(path)
mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=16)
print(f"Original MFCC shape: {mfccs.shape}")
features = mfccs.T
print(f"Transposed MFCC shape: {features.shape}")
timesteps = 129
num_features = 16
if features.shape[0] < timesteps:
    pad_width = timesteps - features.shape[0]
    features = np.pad(features, ((0, pad_width), (0, 0)), mode='constant')
else:
    features = features[:timesteps, :]

# print(f"Padded MFCC shape: {features.shape}")
features = features.reshape(1, timesteps, num_features)
# print(f"Final input shape: {features.shape}")

predictions = model.predict(features)
predicted_label = np.argmax(predictions)
predicted_class = speaker_folders[predicted_label]

print("Predicted class:", predicted_class)

Original MFCC shape: (16, 91)
Transposed MFCC shape: (91, 16)
1/1 [==============================] - 3s 3s/step
Predicted class: YAF_angry
